In [ ]:
#librerie necessarie

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import PyPDFLoader
from dotenv import load_dotenv
from langchain_core.runnables import Runnable, RunnablePassthrough, RunnableParallel, RunnableLambda
from IPython.display import JSON
import json
from unstructured_client import UnstructuredClient
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError
from unstructured.partition.html import partition_html
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import dict_to_elements, elements_to_json


In [ ]:

# Carica le variabili di ambiente dal file .env
load_dotenv()

# Ora puoi accedere alle variabili di ambiente
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
#llm studio

model = ChatOpenAI(
    base_url = "http://172.17.0.1:1234/v1",
    temperature = 0,
    api_key = "not-needed",
    model_name ="llama3",
)

In [ ]:
#ollama

model = ChatOpenAI(
    base_url = "http://localhost:11434/v1",
    temperature = 0,
    api_key = "not-need",
    model_name ="llama3",
)

In [ ]:
#openai

model = ChatOpenAI(
    emperature = 0,
     model_name ="gpt-4o-mini",
)

In [ ]:
pdfd="/home/nugh75/git-repository/env1/Algorithm_of_Thoughts-_Enhancing_Exploration_of_Ideas_in_Large_Language_Models.pdf"

In [ ]:
#Caricamento file da cartella

#percorso della cartella contenente i PDF
folder_path = 'pdfs'

# Lista per memorizzare tutti i documenti caricati
all_documents = []

# Iterare su tutti i file nella cartella
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(folder_path, filename)
        loader = PyPDFLoader(file_path=pdf_path)
        documents = loader.load()
        all_documents.extend(documents)

In [ ]:
len(all_documents)

In [ ]:
print(all_documents)

In [ ]:
print(all_documents[0])

In [ ]:
all_documents[16].metadata

In [ ]:
len(all_documents[0].page_content)

In [ ]:
# Stampare i nomi dei file PDF caricati
print("PDF caricati:")
for pdf in all_documents:
    print(pdf)

In [ ]:
print(all_documents[0].page_content[0:1000])

In [ ]:
all_documents=testo

In [ ]:
# Assumi che pdfd sia già definita e carica i documenti una volta

# Carica il PDF una volta
loader = PyPDFLoader(pdfd)
documents = loader.load()

# Assicurati che ogni documento abbia l'attributo 'page_content'
for doc in documents:
    print(doc.page_content)

# Funzione per formattare i documenti
def format_documents(documents):
    return "\n\n".join(doc.page_content for doc in documents)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =500,
    chunk_overlap=200
)
splits =text_splitter.split_documents(documents)
for sp in splits:
    if (len(sp.page_content) <100):
        splits.remove(sp)

In [ ]:
print ("Numero di chunks:",len(splits))

In [ ]:
print("Metadata :", splits[0].metadata)
print("Dimenzione del chunk:", len(splits[1].page_content), "caratteri")
print("Contenuto del chunk: \n" )
print(splits[1].page_content)

In [ ]:
#Select Embeddings model questo funziona

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

In [ ]:
cartella="algoritmo3"

In [ ]:
import os
from langchain_community.vectorstores import FAISS

if os.path.exists(cartella):
    #Carica indice FAISS cartella corrente attuale
    faiss_index=FAISS.load_local(
        cartella,
        embeddings,
        allow_dangerous_deserialization=True
    )
else:
    #crea indice FAISS dei chunk nella cartella attuale
    faiss_index = FAISS.from_documents(
        splits,
        embeddings
    )
    faiss_index.save_local(cartella)


In [ ]:
retriever = faiss_index.as_retriever(

    search_type="similarity",
    search_kwargs={"k": 4} 
)

In [ ]:
from langchain import hub
prompt =hub.pull("rlm/rag-prompt")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
prompt = ChatPromptTemplate.from_template("Sei un assistente preciso e attento ; Rispondi a questa domanda in italiano: {question}, considra il seguente contesto {context}.")

In [ ]:
#def format_documents(documents):
 #   return "\n\n".join(doc.page_contente for doc in documents)

rag_chain = (
    {
        "context": retriever | format_documents,
        "question": RunnablePassthrough()
        
    } 
    |prompt
    |model
    |StrOutputParser()
    
    
)

In [ ]:
def query(query):
    answer = rag_chain.invoke(query)
    return answer

def queryStream(query):
    for chunk in rag_chain.stream(query):
        print(chunk, end="", flush=True)

In [ ]:
query("che cosa è un prompt")

In [ ]:
queryStream("Dammi esempi di prompt efficaci")